In [33]:
import dask.dataframe as dd
from dask.distributed import Client
import json
import dask
import numpy as np
import re
import graphviz
import pandas as pd

In [9]:
#dask.config.config

In [2]:
client=Client('172.31.47.111:8786')
client
# client.scheduler_info()
# #dask.config.config

Client Scheduler: tcp://172.31.47.111:8786 Dashboard: http://172.31.47.111:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.14 GB


In [3]:
products = dd.read_csv("small_products_Release.csv")
reviews = dd.read_csv("small_user_reviews_Release.csv")

In [34]:
products.head(4)

,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,0001048791,{'Books': 6334800},http://ecx.images-amazon.com/images/I/51MKP0T4...,[['Books']],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,NaN
1,0000143561,{'Movies & TV': 376041},http://g-ecx.images-amazon.com/images/G/01/x-s...,"[['Movies & TV', 'Movies']]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...",12.99,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN
2,0000037214,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN,6.99,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Big Dreams
3,0000032069,NaN,http://ecx.images-amazon.com/images/I/51EzU6qu...,"[['Sports & Outdoors', 'Other Sports', 'Dance'...",Adult Ballet Tutu Cheetah Pink,NaN,7.89,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi


In [13]:
reviews.head(1)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2T0RJ91B0PQ03,B0016CRVLW,Gerald DeWitt,"[0, 0]",Beware! This is NOT the original single versi...,1.0,Poor Quality Alternate Take,1.400630e+09,"05 21, 2014"


In [14]:
products.shape[0].compute()

999999

In [15]:
products.head()


,asin,salesRank,imUrl,categories,title,description,price,related,brand
0,0001048791,{'Books': 6334800},http://ecx.images-amazon.com/images/I/51MKP0T4...,[['Books']],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,NaN
1,0000143561,{'Movies & TV': 376041},http://g-ecx.images-amazon.com/images/G/01/x-s...,"[['Movies & TV', 'Movies']]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...",12.99,"{'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '...",NaN
2,0000037214,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN,6.99,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Big Dreams
3,0000032069,NaN,http://ecx.images-amazon.com/images/I/51EzU6qu...,"[['Sports & Outdoors', 'Other Sports', 'Dance'...",Adult Ballet Tutu Cheetah Pink,NaN,7.89,"{'also_bought': ['0000032050', 'B00D0DJAEG', '...",BubuBibi
4,0000031909,{'Toys & Games': 201847},http://ecx.images-amazon.com/images/I/41xBoP0F...,"[['Sports & Outdoors', 'Other Sports', 'Dance']]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,7.00,"{'also_bought': ['B002BZX8Z6', 'B00JHONN1S', '...",Unknown


In [8]:
percentNa = (reviews.isna().mean().mean()*100 + products.isna().mean().mean()*100)/2
percentNa.compute()

#check mask""???

19.697974757525355

In [18]:
subproducts = products[['asin', 'price']]
subreviews = reviews[['asin', 'overall']]

#optimise with left and not index on products
joined = subproducts.join(subreviews.set_index("asin"), on="asin", how='inner')
#joined.head()
pcor = joined.corr()['overall']
val = pcor.compute()['price']
val

-0.004463768537048657

In [19]:
price_prod = products['price'].describe().compute()
#check if min is actuatly returning empty
[price_prod['mean'], price_prod['std'], price_prod['50%'], price_prod['min'], price_prod['max']]

[22.215322309534532, 33.11203156857402, 16.14, 0.0, 995.95]

In [23]:
def f(x):
    if isinstance(x, float):
        return x
    else:
        return x.strip("[]").split(',')[0]

numCat = products.assign(categories = lambda x: x.categories.apply(
    f,
    meta=('categories', 'str'))
                        )

numCat = numCat.groupby("categories").count()['asin']
numCat.compute()[1:]

#[1:]this removes the nan catagory

categories
'Books'                        992110
'CDs & Vinyl'                     361
'Clothing                          19
'Movies & TV'                    5058
'Musical Instruments'              24
'Software'                        130
'Sports & Outdoors'                57
'Tools & Home Improvement'          7
'Toys & Games'                    678
'Baby'                             14
'Cell Phones & Accessories'         8
'Electronics'                      42
'Health & Personal Care'           29
'Home & Kitchen'                   65
'Office & School Supplies'         26
'Office Products'                 351
'Video Games'                      30
'Automotive'                        3
'Beauty'                            5
'Arts                              70
'Computers'                         3
'All Electronics'                   4
'Grocery & Gourmet Food'            2
'Pet Supplies'                      9
'Industrial & Scientific'           2
'Kitchen & Dining'                  2
'

In [24]:
#Q5

subR = reviews[['asin']].drop_duplicates().set_index('asin')

subP = products.set_index('asin')

iner = subR.join(subP, on="asin", how='inner').shape[0]

left = subR.join(subP, on="asin", how='left').shape[0]

print(iner)
print(left)

out = (iner == left).compute()

out

#products.set_index('asin').index
#index.difference(products.set_index('asin').index)

Delayed('int-78ba916a-495c-4240-ae14-de7c8540c9ab')
Delayed('int-6a25e52d-0efa-4021-bf84-c27a43d9a18a')


False

In [50]:
#Q6

subP = products

ser = subP["asin"]

def parse(related):
    return re.findall("[A-Z0-9]{10}", related)

def f(val):
    asins = parse(val)
    return ser.str.contains("|".join(asins)).any()
    
def g(df):
    df['related'].apply(f,meta="str")

#f({"related":"0000143561"}).compute()
#f({"related":"B0036FO6SI"}).compute()

subP.assign(related = g,meta="str").compute()



,asin,salesRank,imUrl,categories,title,description,price,related,brand,meta
0,0001048791,{'Books': 6334800},http://ecx.images-amazon.com/images/I/51MKP0T4...,[['Books']],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,None,NaN,str
1,0000143561,{'Movies & TV': 376041},http://g-ecx.images-amazon.com/images/G/01/x-s...,"[['Movies & TV', 'Movies']]","Everyday Italian (with Giada de Laurentiis), V...","3Pack DVD set - Italian Classics, Parties and ...",12.99,None,NaN,str
2,0000037214,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,NaN,6.99,None,Big Dreams,str
3,0000032069,NaN,http://ecx.images-amazon.com/images/I/51EzU6qu...,"[['Sports & Outdoors', 'Other Sports', 'Dance'...",Adult Ballet Tutu Cheetah Pink,NaN,7.89,None,BubuBibi,str
4,0000031909,{'Toys & Games': 201847},http://ecx.images-amazon.com/images/I/41xBoP0F...,"[['Sports & Outdoors', 'Other Sports', 'Dance']]",Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,7.00,None,Unknown,str
...,...,...,...,...,...,...,...,...,...,...
55300,0982761457,{'Books': 1568079},http://ecx.images-amazon.com/images/I/71KzQWaT...,[['Books']],The Heebie Jeebies,NaN,8.96,None,NaN,str
55301,0982772025,{'Books': 641666},http://ecx.images-amazon.com/images/I/71mM6xyG...,[['Books']],Things Go Wrong For Me (when life hands you le...,&quot;A hysterical book written by a man with ...,3.99,None,NaN,str
55302,0982776225,{'Books': 118525},http://ecx.images-amazon.com/images/I/4148nxJi...,[['Books']],Write Now The Getty-Dubay Program for Handwrit...,NaN,18.95,None,NaN,str
55303,0982774370,{'Books': 788370},http://ecx.images-amazon.com/images/I/81QgBnwe...,[['Books']],The Mighty Dead,NaN,9.99,None,NaN,str


In [35]:
subP = products.set_index("asin")


def parse(related):
    return re.findall("[A-Z0-9]{10}", related)

def f(df):
    asins = []
    for row in df.iterrows():
        
        asins.append([1])
    
    ret = pd.DataFrame({"asins" : asins})
    return ret
    

subP.map_partitions(f,  meta="str").compute()


AttributeError: 'Series' object has no attribute 'columns'